In [1]:
# !pip install transformers
# !pip install accelerate


import os
# os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer
from tqdm import tqdm
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
data = pd.read_csv('./data/train.csv')

In [4]:

# tokenizer = AutoTokenizer.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   eos_token='</s>'
# )
model = AutoModelForCausalLM.from_pretrained("./kakao_6_9_epoch/")
tokenizer = AutoTokenizer.from_pretrained("./kakao_6_9_epoch/", eos_token='</s>')


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
formatted_data = []
for _, row in tqdm(data.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            # 질문과 답변 쌍을 </s> token으로 연결
            input_text = row[q_col] + tokenizer.eos_token + row[a_col] + tokenizer.eos_token
            input_ids = tokenizer.encode(input_text, return_tensors='pt')
            formatted_data.append(input_ids)

644it [00:02, 269.54it/s]


In [6]:
# for name, module in model.named_children():
#     if name != "lm_head":
#         for param in module.parameters():
#             param.requires_grad = False

In [7]:
model

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(64512, 4096)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f)

In [8]:
for name, module in model.named_children():
    if name == "transformer":
        for name, sub_module in module.named_children():
            # print(name)
            if name == "wte":
                for param in sub_module.parameters():
                    param.requires_grad = False
            elif name == "drop":
                for param in sub_module.parameters():
                    param.requires_grad = False
            elif name == "h":
                # print(name)
                 for name, layer_module in sub_module.named_children():
                     if int(name) < 20:
                        for param in layer_module.parameters():
                            param.requires_grad = False
            

In [9]:
CFG = {
    'LR' : 5e-6, # Learning Rate
    'EPOCHS' : 10, # 학습 Epoch
}
# for name, module in model.named_children():
#     if name != "lm_head":
#         for param in module.parameters():
#             param.requires_grad = False
    
model.to(device) # 모델을 GPU단으로 이동

# 모델 학습 설정
optimizer = AdamW(model.parameters(), lr=CFG['LR'])
model.train()

# 모델 학습
for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    progress_bar = tqdm(enumerate(formatted_data), total=len(formatted_data))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")

    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(formatted_data)}")
    model.save_pretrained(f"./kakao_7_{epoch}_epoch")
    tokenizer.save_pretrained(f"./kakao_7_{epoch}_epoch")
# model.save_pretrained("./kakao_10epoch")
# tokenizer.save_pretrained("./kakao_10epoch")

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1 - Avg Loss: 0.1317:   0%|▎                                                                                                                                          | 12/6440 [00:06<1:01:20,  1.75it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1008.00 MiB (GPU 0; 47.54 GiB total capacity; 44.23 GiB already allocated; 824.25 MiB free; 45.58 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF